In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Jak tabulky interpretovat

Každá potenciální odpověď má jeden řádek.

### Correct/All evals ratio

- by ideálně mělo být 100 %
- indikuje to, v kolika procentech odevzání byla daná možnost správně (ne)zaškrtnutá
    - i.e. 100 % by znamenalo, že pokud je to
        - správná odpověď, tak byla ve všech odevzdáních zaškrtnuta
        - špatná odpověď, tak nebyla v žádném odevzdání zaškrtnuta
- potenciální problém: jeden řešitel, který neví a tipuje, má větší efekt na procento než řešitel, který to dá na první pokus správně

**Důsledky pro seminář**:

- odpovědi, které mají nízké procento (třeba méně než < 80 %), jsou "chytáky", nebo potenciálně špatně vysvětlené


### Count evals

- absolutní počet odevzdání, které měli danou odpověď zaškrtnutou
- pro správné odpovědi je podkreslení zelené
- pro špatné odpovědi je podkreslení červené
- potenciální problém stejný jako pro předchozí sloupec: jeden řešitel, který neví a tipuje, má větší efekt na procento než řešitel, který to dá na první pokus správně


### Ticked at least once/All users ratio

- Procento řešitelů, kteří odpověď zaškrtli alespoň jednou, ze všech řešitelů, kteří modul řešili.
- Pro špatné odpovědi bychom ideálně chtěli 0 %, pro správně odpovědi ideálně 100 %.
- 100 % na správné odpovědi je špatně dosažitelných, stačí jeden řešitel jež úlohu odevzdal neúspěšně a pak ji nikdy úspěšně nedokončil.

**Důsledky pro seminář**:

Špatné odpovědi by měly mít malé procento, pokud nemají, tak jsou to chytáky, nebo špatně vysvětlené.

Správné odpovědi by měli mít velké procento, pokud nemají, tak jsme nejspíš nedostatečně vysvětlili, proč je tvrzení pravdivé.

Opakovaně neúspěšní/tipující řešitelé nemají takový vliv na výsledná procenta.


In [ ]:
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
from collections import OrderedDict, namedtuple
from sqlalchemy import func, distinct, text, and_
import pandas as pd
from IPython.display import display, HTML, Markdown
import seaborn as sns
import numpy as np

import util
from util.year import year as current_year
from db import session
import model
from datetime import datetime
import re
import json
from collections import Counter

pd.options.display.float_format = '{:.2f}'.format
plt.rcParams['figure.figsize'] = [8, 6]
print(datetime.now())

In [ ]:
evaluations = session.query(
    model.EvaluationParticipantsWithContext
)\
.filter(model.EvaluationParticipantsWithContext.year_id == current_year.id)\
.filter(model.EvaluationParticipantsWithContext.type == "quiz")\
.order_by(model.EvaluationParticipantsWithContext.task_id, model.EvaluationParticipantsWithContext.module_id)

In [ ]:
out = evaluations.all()

In [ ]:
Answer = namedtuple("Answer", ["evaluation", "user_answers", "correct_answers"])
Stats = namedtuple("Stats", ["evaluation", "correct_answer", "combination_counts", "item_counts", "item_users", "total_evaluations", "users"])


RE_ANSWER_LINE = re.compile(r"^\s*\[[yn]\] Question \d+ -- user answers: \[(.*)\], correct answers: \[(.*)\]")


def answer_to_list(answer):
    return tuple() if not answer else tuple(int(a) for a in answer.split(", "))

def extract_answer(evaluation):
    lines = evaluation.full_report.split("\n")
    user_answers = []
    correct_answers = []
    for line in lines:
        if line.strip().startswith("["):
            match = RE_ANSWER_LINE.match(line)
            assert match
            user_answers.append(answer_to_list(match.group(1)))
            correct_answers.append(answer_to_list(match.group(2)))
    return Answer(evaluation, tuple(user_answers), tuple(correct_answers))

def extract_stats(evaluations):
    answers = [extract_answer(e) for e in evaluations]

    result = []
    prev_module_id = None
    for i, evaluation in enumerate(evaluations):
        if prev_module_id is None or prev_module_id != evaluation.module_id:
            result.append(Stats(evaluation, answers[i].correct_answers, Counter(), Counter(), {}, Counter(), set()))
            prev_module_id = evaluation.module_id
        user_answers = answers[i].user_answers
        result[-1].combination_counts.update([user_answers])
        result[-1].item_counts.update([(item, answer_item) for item, answer in enumerate(user_answers) for answer_item in answer])
        for item, answer in enumerate(user_answers):
            for answer_item in answer:
                result[-1].item_users[(item, answer_item)] = result[-1].item_users.get((item, answer_item), set()) | {evaluation.user}
        result[-1].total_evaluations.update(["total"])
        result[-1].users.add(evaluation.user)
    return result


stats = extract_stats(out)

In [ ]:
def highlight_by_question(ignore_columns):
    color_switch = True
    def inner(row):
        nonlocal color_switch
        color_switch = (not color_switch) if row["Question"] else color_switch
        base = [f"background-color: {'#F9F9F9' if color_switch else 'white'}"] * len(row)
        for i in ignore_columns:
            base[i] = "background-color: white; border-right: 1px solid #dddddd"
        return base
    return inner

def make_header(ev):
    display(Markdown(f"## {ev.task_id} {ev.task_name}\n### {ev.module_id} {ev.module_name}"))

    
GREEN = "#5fd65f"
RED = "#FF9393"
def is_correct_answer(row):
    return row["Correct"].lower() == "yes"

def color_by_correctness(cols, stat):
    def inner(row):
        styles = ["" for _ in range(len(row))]
        for id_ in cols:
            styles[id_] = "background-color: " + (GREEN if is_correct_answer(row) else RED)
        return styles
    return inner

def custom_bar(col_index, vmin, vmax):
    def inner(row):
        base = ["" for _ in range(len(row))]
        val = row.iloc[col_index]
        ratio = f"{(val - vmin)/vmax * 100:.1f}"
        base[col_index] = f"border-left: 1px solid; width: 10em; background: linear-gradient(90deg,{GREEN if is_correct_answer(row) else RED} {ratio}%, transparent {ratio}%);"
        return base
    return inner

def custom_gradient(col_index, cmap, ondra=False):
    def get_hex_color(val, cmap):
        r, g, b, a = cmap(val)
        return f"rgb({r * 255}, {g * 255}, {b * 255})"

    def inner(row):
        base = ["" for _ in range(len(row))]
        val = row.iloc[col_index]
        if not ondra:
            ratio = val / 100
        else:
            ratio = 0.5 - val / 200 * (-1 if is_correct_answer(row) else 1)
        base[col_index] = f"background-color: {get_hex_color(ratio, cmap)}!important;"
        if not ondra and val > 85:
            base[col_index] += "color: white!important"
        return base
    return inner

def show_evaluations_stats(evaluation_stats):
    module_data = {
        id: json.loads(data)["quiz"] for id, data in 
            session.query(model.Module.id, model.Module.data).filter(model.Module.type == model.ModuleType.QUIZ).all()
    }
    
    for stat in evaluation_stats:
        this_module_data = module_data[stat.evaluation.module_id]
        data = []
        total_evaluations = stat.total_evaluations["total"]
        total_users = len(stat.users)

        for answer, count in sorted(stat.item_counts.items()):
            item_i, user_a = answer
            ticked_ratio = count / total_evaluations
            ticked_ratio_user = len(stat.item_users[(item_i, user_a)]) / total_users
            try:
                data.append((
                    item_i,
                    user_a,
                    this_module_data[item_i]["question"] if not data or data[-1][0] != item_i else "",
                    this_module_data[item_i]["options"][user_a],
                    ticked_ratio * 100,
                    (ticked_ratio if user_a in stat.correct_answer[item_i] else 1 - ticked_ratio) * 100,
                    "Yes" if user_a in stat.correct_answer[item_i] else "No",
                    count,
                    ticked_ratio_user * 100
                ))
            except IndexError:
                data.append((
                    item_i,
                    user_a,
                    "ERROR: some question in this module probably removed",
                    "ERROR: some question in this module probably removed",
                    ticked_ratio * 100,
                    (ticked_ratio if user_a in stat.correct_answer[item_i] else 1 - ticked_ratio) * 100,
                    "Yes" if user_a in stat.correct_answer[item_i] else "No",
                    count,
                    ticked_ratio_user * 100
                ))
        
        df = pd.DataFrame(data, columns=[
            "Item ID",
            "Answer ID",
            "Question",
            "Answer",
            "Ticked in/All evals ratio",
            "Correct/All evals ratio",
            "Correct",
            "Count evals",
            "Ticked at least once/All users ratio",
#             "Count users",
        ])

        s = df.style.hide_columns(["Item ID", "Answer ID", "Correct", "Ticked in/All evals ratio"])
        s.apply(custom_bar(col_index=7, vmin=0, vmax=total_evaluations), axis=1)
        
        s.format({
            'Ticked in/All evals ratio': '{:,.1f} %'.format,
            'Correct/All evals ratio': '{:,.1f} %'.format,
            'Ticked at least once/All users ratio': '{:,.1f} %'.format,
        })
        
        s.apply(highlight_by_question(ignore_columns=[7]), axis=1)
        s.apply(custom_gradient(col_index=5, cmap=plt.get_cmap("RdYlGn")), axis=1)
        s.apply(custom_gradient(col_index=5, cmap=plt.get_cmap("RdYlGn")), axis=1)
        s.apply(custom_gradient(col_index=8, cmap=sns.diverging_palette(20, 127, s=78, l=77, as_cmap=True), ondra=True), axis=1)
#         s.apply(custom_gradient(col_index=4, cmap=sns.diverging_palette(20, 120, l=75, as_cmap=True)), axis=1)
        make_header(stat.evaluation)
        display(s)

show_evaluations_stats(stats)